In [1]:
import pandas as pd
from functions import check_nan
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
actors = pd.read_csv("../data/cleaned/actors_clean.csv")
films = pd.read_csv("../data/cleaned/films_clean.csv")
old_HDD = pd.read_csv("../data/cleaned/old_HDD_clean.csv")


I need to create a database for the common table for actors and films on mySQL. For that, I will use the actors, films and old_HDD data. This is the only way I can join them since the actor and old_HDD share first_name and last_name and film and old_HDD share title. I need to create actors_films and it needs to include actor_id (in actor) and film_id (in film)

In [3]:
actors_subset = actors[['first_name', 'last_name','actor_id']]
films_subset = films[['title','film_id']]

actorss_filmss = old_HDD.merge(actors_subset, on=['first_name', 'last_name'], how='left')
actorss_filmss = actorss_filmss.merge(films_subset, on='title', how='left')




In [4]:
actorss_filmss.head()

,first_name,last_name,title,category_id,actor_id,film_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,6,1,1
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2,1,23
2,PENELOPE,GUINESS,ANGELS LIFE,13,1,25
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,10,1,106
4,PENELOPE,GUINESS,CHEAPER CLYDE,14,1,140


In [5]:
selected_columns=['actor_id','film_id']
actors_films=actorss_filmss[selected_columns]
actors_films.head()

,actor_id,film_id
0,1,1
1,1,23
2,1,25
3,1,106
4,1,140


I merged the three and selected only the columns that I was interested in. 

In [6]:
actors_films.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   actor_id  1000 non-null   int64
 1   film_id   1000 non-null   int64
dtypes: int64(2)
memory usage: 23.4 KB


In [7]:
for c in actors_films.select_dtypes(include="integer"):
    
    actors_films[c]=pd.to_numeric(actors_films[c], downcast="integer")

Downcasted integers

In [8]:
actors_films.to_csv("../data/cleaned/actors_films_clean.csv", index=False)


Saved new file

In [9]:
films.shape

(1000, 9)

In [10]:
films_withcategory = pd.merge(films, old_HDD, on=['title'], how='left')
films_withcategory.tail()


,film_id,title,description,rental_duration_days,rental_rate,length,replacement_cost,rating,special_features,first_name,last_name,category_id
1381,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,4,0.99,179,14.99,NC-17,"Trailers,Behind the Scenes",SANDRA,KILMER,12.0
1382,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,4,0.99,179,14.99,NC-17,"Trailers,Behind the Scenes",VAL,BOLGER,12.0
1383,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,6,0.99,105,10.99,NC-17,Deleted Scenes,UMA,WOOD,11.0
1384,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,5,2.99,101,28.99,R,"Trailers,Deleted Scenes",NaN,NaN,NaN
1385,1000,ZORRO ARK,A Intrepid Panorama of a Mad Scientist And a B...,3,4.99,50,18.99,NC-17,"Trailers,Commentaries,Behind the Scenes",NaN,NaN,NaN


I also have to put category_id on films. I will again use old_HDD for this

In [11]:
films_withcategory=films_withcategory[["film_id", "title", "description", "rental_duration_days", "rental_rate", "length", "replacement_cost", "rating", "special_features", "category_id"]]

Selecting only the columns I am interested in. 

In [12]:
films_withcategory.duplicated().any()

True

In [13]:
films_withcategory=films_withcategory.drop_duplicates()

In [14]:
films_withcategory.shape

(1000, 10)

Dropped duplicates.

In [15]:
films_withcategory.isnull().sum()

film_id                   0
title                     0
description               0
rental_duration_days      0
rental_rate               0
length                    0
replacement_cost          0
rating                    0
special_features          0
category_id             386
dtype: int64

In [16]:
films_withcategory.fillna(value=17, inplace=True)

There were only 614 rows in old_HDD but 1000 in film. When I merged them to add category_id in films, I created 386 null values. I filled those in with "unknown". 

In [17]:
films_withcategory.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 1385
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   rental_duration_days  1000 non-null   int64  
 4   rental_rate           1000 non-null   float64
 5   length                1000 non-null   int64  
 6   replacement_cost      1000 non-null   float64
 7   rating                1000 non-null   object 
 8   special_features      1000 non-null   object 
 9   category_id           1000 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 85.9+ KB


In [18]:
films_withcategory["category_id"] = films_withcategory["category_id"].astype("int")

In [19]:
for c in films_withcategory.select_dtypes(include="integer"):
    
    films_withcategory[c]=pd.to_numeric(films_withcategory[c], downcast="integer")

In [20]:
for c in films_withcategory.select_dtypes(include="float"):
    
    films_withcategory[c]=pd.to_numeric(films_withcategory[c], downcast="float")

In [21]:
films_withcategory["rating"] = films_withcategory["rating"].astype("category")

In [22]:
films_withcategory.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 1385
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   film_id               1000 non-null   int16   
 1   title                 1000 non-null   object  
 2   description           1000 non-null   object  
 3   rental_duration_days  1000 non-null   int8    
 4   rental_rate           1000 non-null   float32 
 5   length                1000 non-null   int16   
 6   replacement_cost      1000 non-null   float32 
 7   rating                1000 non-null   category
 8   special_features      1000 non-null   object  
 9   category_id           1000 non-null   int8    
dtypes: category(1), float32(2), int16(2), int8(2), object(3)
memory usage: 46.1+ KB


Downcasted integers and floats and changed rating type to category

In [23]:
films_withcategory.to_csv("../data/cleaned/films_category_clean.csv", index=False)


Saved the file